In [1]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
import pandas as pd 

sql = """
select icscode as icscode_depart, station_name as station_name_depart from tt_h.parsed_rail_journeys
where natrail_journey_minutes is null
"""

problem_stations = pd.read_sql(sql, conn)
problem_stations["join"] = 1

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# Get list of all cl stations 
sql = """
SELECT station_name as station_name_arrive, icscode as icscode_arrive, "1516_entries_&_exits"
from tt_h.all_stations as s, tt_h.inner_london as ie
where ST_Contains(ie.geom, s.geom)
order by cast("1516_entries_&_exits" as float) desc
limit 10
"""

cl_stations = pd.read_sql(sql, conn)
cl_stations["join"] = 1

In [3]:
to_lookup = pd.merge(problem_stations, cl_stations, on="join")[["icscode_depart", "station_name_depart", "icscode_arrive", "station_name_arrive"]]

In [4]:
# to_lookup.to_sql("anomalies_tfl_json", engine, schema="tt_h", if_exists="replace", index=False)

In [5]:
# sql = """
# ALTER TABLE tt_h.anomalies_tfl_json ADD COLUMN id SERIAL PRIMARY KEY;

# ALTER TABLE tt_h.anomalies_tfl_json 
#  ADD tfl_request text, 
#  ADD tfl_response json,
#  ADD querydict json,
#  ADD tfl_message text;

# """
# cursor.execute(sql)
# conn.commit()

In [6]:
pd.read_sql("select * from tt_h.anomalies_tfl_json  ", conn).head(1)

,icscode_depart,station_name_depart,icscode_arrive,station_name_arrive,id,tfl_request,tfl_response,querydict,tfl_message
0,90002286,Dover Priory,1000139,London Bridge,274,https://api.tfl.gov.uk/Journey/JourneyResults/...,"{""$type"":""Tfl.Api.Presentation.Entities.Journe...","{""to"": ""1000139"", ""id"": ""cedea352"", ""key"": ""39...",ok


In [8]:
# Now 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
import utils
from mylibrary.tfl_helpers import get_journeyplanner_results
Journeys = Automapped_Base.classes.anomalies_tfl_json

sql = """
select id
from tt_h.anomalies_tfl_json
where tfl_message != 'ok' or tfl_response is null
"""

ids = pd.read_sql(sql, conn)
ids = [int(i) for i in ids["id"]]
print(len(ids))


250


In [9]:

counter = 0
for id in ids:
    counter +=1
    journey = session.query(Journeys).filter(Journeys.id == id).one()

    attempts = utils.get_attempt(journey.icscode_arrive, ["0800","0820", "0840"], ["20170817","20170906","20170908"])
    for a in attempts:
        a["from"] = journey.icscode_depart

    journey_object = get_journeyplanner_results(attempts)

    journey.tfl_request = journey_object["request_url"]
    journey.tfl_response = journey_object["tfl_response"]
    journey.querydict = journey_object["query_dict"]
    journey.tfl_message = journey_object["tfl_message"]


    session.add(journey)

    session.commit()
    
    if counter % 5 == 0:
        print(counter)
    


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
